# Cardinal Restaurant Project

## Importing Libraries

In [55]:
import tkinter as tk
from tkinter import ttk, messagebox
from confidential import api_key
import pandas as pd
import os
from tkinter import messagebox
from decouple import config
from groq import Groq

## Input Data From Spreadsheet

In [322]:
class RestaurantPOS:
    def __init__(self, root, csv_file):        
        self.root = root
        self.csv_file = csv_file
        self.root.title("Cardinal Restaurant POS")
        self.cart = []

        # Load menu data
        self.menu_data = pd.read_csv(self.csv_file)

        # Configure window layout and style
        self.root.geometry("1000x600")  # Adjust size
        self.root.configure(bg="lightgrey")
        
        # Configure grid layout
        self.root.grid_columnconfigure(0, weight=1)  # Chatbot
        self.root.grid_columnconfigure(1, weight=2)  # Menu
        self.root.grid_columnconfigure(2, weight=1)  # Cart

        # Create header with dark gray box
        self.create_header()

        # Create restaurant selection
        self.create_restaurant_selection()

        # Create cart and chatbot area
        self.create_chatbot_area()
        self.create_cart_area()

        # Hide the cart area initially
        self.cart_frame.grid_forget()

        # Store the current restaurant and category for navigation
        self.current_restaurant = None
        self.current_category = None

    def create_header(self):
        # Create a dark gray header that spans across the top
        header_frame = tk.Frame(self.root, bg="darkgray", height=50)
        header_frame.pack(fill=tk.X)

        # Add the Cardinal Restaurant label aligned to the left
        header_label = tk.Label(
            header_frame, text="Cardinal Restaurant",
            font=("Helvetica", 20, "bold"), fg="red", bg="darkgray"
        )
        header_label.pack(side=tk.LEFT, padx=20, pady=10)

        # Create and pack the cart button inside the header (right side)
        self.cart_button = tk.Button(
            header_frame, text="🛒", font=("Helvetica", 16), bg="lightgreen", activebackground="green",
            command=self.toggle_cart
        )
        self.cart_button.pack(side=tk.RIGHT, padx=10, pady=10)

    def create_restaurant_selection(self):
        # Get unique restaurant names
        restaurants = self.menu_data['restaurant'].unique()
        
        # Create the selection frame
        self.restaurant_frame = tk.Frame(self.root, bg="lightgrey")
        self.restaurant_frame.pack(pady=20)
        
        label = tk.Label(
            self.restaurant_frame, text="Select a Restaurant:",
            font=("Helvetica", 16), bg="lightgrey"
        )
        label.pack(pady=10)
        
        # Dynamically create buttons for each restaurant
        for restaurant in restaurants:
            button = tk.Button(
                self.restaurant_frame, text=restaurant,
                font=("Helvetica", 14), bg="lightgreen", activebackground="green",
                command=lambda r=restaurant: self.load_menu(r)
            )
            button.pack(pady=5)
    
    def load_menu(self, restaurant_name):
        self.current_restaurant = restaurant_name
        self.restaurant_frame.destroy()
        self.menu_frame = tk.Frame(self.root, bg="lightgrey")
        self.menu_frame.pack(pady=20)
        
        label = tk.Label(
            self.menu_frame, text=f"Menu - {restaurant_name}",
            font=("Helvetica", 16, "bold"), bg="lightgrey"
        )
        label.pack(pady=10)
        
        # Create a back button to go back to the restaurant selection
        back_to_restaurants_button = tk.Button(
            self.menu_frame, text="Back to Restaurants",
            font=("Helvetica", 12), bg="lightgrey", activebackground="grey",
            command=self.back_to_restaurants
        )
        back_to_restaurants_button.pack(pady=5)
        
        # Filter menu data by the selected restaurant
        restaurant_data = self.menu_data[self.menu_data["restaurant"] == restaurant_name]
        categories = restaurant_data["category"].unique()
        
        for category in categories:
            category_button = tk.Button(
                self.menu_frame, text=category,
                font=("Helvetica", 14), bg="lightgreen", activebackground="green",
                command=lambda c=category: self.load_items(c, restaurant_name)
            )
            category_button.pack(pady=5)
    
    def load_items(self, category, restaurant_name):
        self.current_category = category
        for widget in self.menu_frame.winfo_children():
            widget.destroy()

        # Create a canvas widget
        self.canvas = tk.Canvas(self.menu_frame, bg="lightgrey")
        self.canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        # Create a scrollbar for the canvas
        scrollbar = tk.Scrollbar(self.menu_frame, orient="vertical", command=self.canvas.yview)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        # Configure the canvas to work with the scrollbar
        self.canvas.configure(yscrollcommand=scrollbar.set)

        # Create a frame inside the canvas to hold the menu items
        self.items_frame = tk.Frame(self.canvas, bg="lightgrey")
        self.canvas.create_window((0, 0), window=self.items_frame, anchor="nw")

        # Add the category title label
        label = tk.Label(
            self.items_frame, text=f"Items - {category}",
            font=("Helvetica", 16, "bold"), bg="lightgrey"
        )
        label.pack(pady=10)

        # Filter items for the selected category and restaurant
        items = self.menu_data[(self.menu_data["category"] == category) & 
                               (self.menu_data["restaurant"] == restaurant_name)]

        # Add buttons for each item in the category
        for _, row in items.iterrows():
            item_button = tk.Button(
                self.items_frame, text=f"{row['item']} (${row['price']:.2f})",
                font=("Helvetica", 12), bg="lightgreen", activebackground="green",
                command=lambda r=row: self.add_to_cart(r)
            )
            item_button.pack(pady=5)

        # Add a back button to go back to categories
        back_button = tk.Button(
            self.items_frame, text="Back to Categories",
            font=("Helvetica", 12), bg="lightgrey", activebackground="grey",
            command=self.back_to_categories
        )
        back_button.pack(pady=10)

        # Update the scrollable region of the canvas
        self.items_frame.update_idletasks()  # Ensure the items are fully rendered
        self.canvas.config(scrollregion=self.canvas.bbox("all"))

        # Bind mouse wheel event to canvas for scrolling
        self.canvas.bind_all("<MouseWheel>", self.on_mouse_wheel)

    def on_mouse_wheel(self, event):
        # Check if the canvas is still valid before attempting to scroll
        if hasattr(self, 'canvas') and self.canvas.winfo_exists():
            self.canvas.yview_scroll(int(-1 * (event.delta / 120)), "units")

  
    
    def back_to_categories(self):
        self.menu_frame.destroy()
        self.load_menu(self.current_restaurant)
    
    def back_to_restaurants(self):
        self.menu_frame.destroy()
        self.restaurant_frame = tk.Frame(self.root, bg="lightgrey")
        self.restaurant_frame.pack(pady=20)
        
        label = tk.Label(
            self.restaurant_frame, text="Select a Restaurant:",
            font=("Helvetica", 16), bg="lightgrey"
        )
        label.pack(pady=10)
        
        # Dynamically create buttons for each restaurant
        restaurants = self.menu_data['restaurant'].unique()
        for restaurant in restaurants:
            button = tk.Button(
                self.restaurant_frame, text=restaurant,
                font=("Helvetica", 14), bg="lightgreen", activebackground="green",
                command=lambda r=restaurant: self.load_menu(r)
            )
            button.pack(pady=5)
    
    def add_to_cart(self, item_row):
        self.cart.append(item_row)
        self.update_cart_display()

        # Update cart button to reflect total price
        total_price = sum(item['price'] for item in self.cart)
        self.cart_button.config(text=f"🛒 ${total_price:.2f}")

    def toggle_cart(self):
        if self.cart_frame.winfo_ismapped():
            self.cart_frame.pack_forget()
        else:
            self.cart_frame.pack(side=tk.RIGHT, padx=10, pady=10)

            # Update cart button to reflect total price when cart is open
            total_price = sum(item['price'] for item in self.cart)
            self.cart_button.config(text=f"🛒 ${total_price:.2f}")

            # Create the remove button only once when the cart is opened
            if not hasattr(self, 'remove_button'):  # Check if remove button exists
                self.remove_button = tk.Button(
                    self.cart_frame, text="Remove Item",
                    command=self.remove_selected_item
                )
                self.remove_button.pack(pady=10)  # Place the button inside the cart frame

            # Update the cart display
            self.update_cart_display()

    def update_cart_display(self):
        if self.cart_frame:
            self.cart_list.delete(0, tk.END)  # Clear existing list

            total_price = 0

            # Add items to the cart list
            for item in self.cart:
                display_text = f"{item['item']} - ${item['price']:.2f}"
                self.cart_list.insert(tk.END, display_text)
                total_price += item["price"]

            # Update total price
            self.total_label.config(text=f"Total: ${total_price:.2f}")

    def remove_selected_item(self):
        # Check if a selection is made in the cart list
        selected_index = self.cart_list.curselection()
        if selected_index:
            selected_index = selected_index[0]  # Get the first selected item
            del self.cart[selected_index]
            self.update_cart_display()

    def create_cart_area(self):
        self.cart_frame = tk.Frame(self.root, bg="lightgrey", height=400)

        cart_label = tk.Label(
            self.cart_frame, text="Cart",
            font=("Helvetica", 16, "bold"), bg="lightgrey"
        )
        cart_label.pack(pady=5)
        
        self.cart_list = tk.Listbox(self.cart_frame, width=40, height=10, font=("Helvetica", 12))
        self.cart_list.pack(pady=5)
        
        self.total_label = tk.Label(self.cart_frame, text="Total: $0.00", font=("Helvetica", 14), bg="lightgrey")
        self.total_label.pack(pady=5)
        
        checkout_button = tk.Button(
            self.cart_frame, text="Checkout",
            font=("Helvetica", 14), bg="lightgreen", activebackground="green",
            command=self.checkout
        )
        checkout_button.pack(pady=10)
    
    def checkout(self):
        if not self.cart:
            messagebox.showinfo("Checkout", "Your cart is empty!")
            return

        # Create a new window for student ID and checkout
        checkout_window = tk.Toplevel(self.root)
        checkout_window.title("Checkout")
        checkout_window.geometry("400x300")
        checkout_window.configure(bg="lightgrey")

        # Student ID entry label
        label = tk.Label(
            checkout_window, text="Enter your Student ID:",
            font=("Helvetica", 14), bg="lightgrey"
        )
        label.pack(pady=10)

        # Student ID entry field
        student_id_entry = tk.Entry(checkout_window, font=("Helvetica", 12))
        student_id_entry.pack(pady=10)

        # Function to process checkout
        def process_checkout():
            student_id = student_id_entry.get()
            if not student_id.strip():
                messagebox.showerror("Error", "Student ID cannot be empty!")
                return

            # Calculate prices
            total_cost = sum(item["price"] for item in self.cart)
            tax = total_cost * 0.06
            total_price = total_cost + tax

            # Display the price breakdown
            items = "\n".join([f"{item['item']} - ${item['price']:.2f}" for item in self.cart])
            messagebox.showinfo(
                "Payment Successful",
                f"Student ID: {student_id}\n\n"
                f"Items:\n{items}\n\n"
                f"Cost: ${total_cost:.2f}\n"
                f"Tax (6%): ${tax:.2f}\n"
                f"Total: ${total_price:.2f}"
            )
            self.cart = []  # Clear the cart
            self.update_cart_display()  # Update cart display
            self.toggle_cart()  # Close the cart display
            checkout_window.destroy()  # Close the checkout window

        # Button to process the checkout
        checkout_button = tk.Button(
            checkout_window, text="Checkout",
            font=("Helvetica", 14), bg="lightgreen", activebackground="green",
            command=process_checkout
        )
        checkout_button.pack(pady=20)

        # Close button
        close_button = tk.Button(
            checkout_window, text="Cancel",
            font=("Helvetica", 14), bg="lightgrey", activebackground="grey",
            command=checkout_window.destroy
        )
        close_button.pack(pady=10)


    def process_user_input(self, user_message):
        # Example input: "I want a meal between 5-7 dollars" or "I want to eat chicken"
        user_message = user_message.lower()

        # Check for price range
        if "between" in user_message and "dollars" in user_message:
            try:
                # Extract the price range from the input (e.g., 5-7)
                price_range = [float(x) for x in user_message.split() if x.replace('.', '', 1).isdigit()]
                if len(price_range) == 2:
                    return self.get_items_by_price_range(price_range[0], price_range[1])

            except ValueError:
                return "Sorry, I couldn't understand the price range. Can you try again?"

        # Check for food type (e.g., chicken, drink)
        if "chicken" in user_message:
            return self.get_items_by_food_type("chicken")

        if "drink" in user_message:
            return self.get_items_by_category("Drink")
        
        if "steak" in user_message:
            return self.get_items_by_food_type("steak")
        
        if "smoothie" in user_message:
            return self.get_items_by_food_type("smoothie")
        
        if "mac and cheese" in user_message:
            return self.get_items_by_food_type("mac and cheese")
        
        if "salad" in user_message:
            return self.get_items_by_food_type("salad")
        
        if "rice" in user_message:
            return self.get_items_by_food_type("rice")

        # Fallback response
        return "I couldn't understand your request. Could you please clarify?"

    def get_items_by_price_range(self, min_price, max_price):
        # Filter menu items by price range
        filtered_items = self.menu_data[(self.menu_data["price"] >= min_price) & (self.menu_data["price"] <= max_price)]

        if filtered_items.empty:
            return "Sorry, I couldn't find any items within that price range."

        response = "Here are some options within your price range:\n"
        for _, row in filtered_items.iterrows():
            response += f"{row['item']} - ${row['price']:.2f}\n"
        return response

    def get_items_by_food_type(self, food_type):
        # Filter items based on the food type (e.g., chicken)
        filtered_items = self.menu_data[self.menu_data['item'].str.contains(food_type, case=False)]

        if filtered_items.empty:
            return f"Sorry, I couldn't find any items containing {food_type}."

        response = f"Here are some {food_type} options:\n"
        for _, row in filtered_items.iterrows():
            response += f"{row['item']} - ${row['price']:.2f}\n"
        return response

    def get_items_by_category(self, category):
        # Filter items by category (e.g., Drinks)
        filtered_items = self.menu_data[self.menu_data['category'].str.contains(category, case=False)]

        if filtered_items.empty:
            return f"Sorry, I couldn't find any {category} items."

        response = f"Here are some {category} options:\n"
        for _, row in filtered_items.iterrows():
            response += f"{row['item']} - ${row['price']:.2f}\n"
        return response

    def create_chatbot_area(self):
        chatbot_frame = tk.Frame(self.root, bg="lightgrey")
        chatbot_frame.pack(side=tk.LEFT, padx=10, pady=10, fill=tk.BOTH, expand=True)
        
        chatbot_label = tk.Label(
            chatbot_frame, text="Chatbot Assistant",
            font=("Helvetica", 16, "bold"), bg="lightgrey"
        )
        chatbot_label.pack(pady=5)
        
        self.chat_display = tk.Text(
            chatbot_frame, wrap=tk.WORD, font=("Helvetica", 12), height=20, width=40, state=tk.DISABLED
        )
        self.chat_display.pack(pady=5)
        
        chat_entry_frame = tk.Frame(chatbot_frame, bg="lightgrey")
        chat_entry_frame.pack(fill=tk.X, padx=5)
        
        self.chat_input = tk.Entry(chat_entry_frame, font=("Helvetica", 12))
        self.chat_input.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=5)
        
        send_button = tk.Button(
            chat_entry_frame, text="Send", font=("Helvetica", 12), bg="lightgreen",
            activebackground="green", command=self.send_chat_message
        )
        send_button.pack(side=tk.RIGHT, padx=5)
    
    def send_chat_message(self):
        user_message = self.chat_input.get()
        if not user_message.strip():
            return

        # Display user message
        self.chat_display.config(state=tk.NORMAL)
        self.chat_display.insert(tk.END, f"You: {user_message}\n")
        self.chat_input.delete(0, tk.END)  # Clear the input field

        # Process the user's message and get the bot's response
        bot_response = self.process_user_input(user_message)

        # Display bot response
        self.chat_display.insert(tk.END, f"Bot: {bot_response}\n\n")
        self.chat_display.config(state=tk.DISABLED)

        # Scroll to the bottom of the chat display
        self.chat_display.yview(tk.END)

In [323]:
root = tk.Tk()
root.title("Restaurant POS System")
pos = RestaurantPOS(root, "menus/menus.csv")
root.mainloop()